In [1]:
import os
import sys
import json

import torch
import torch.nn as nn
from torchvision import transforms, datasets
import torch.optim as optim
from tqdm import tqdm

from model.vgg import vgg

/home/yuchen/anaconda3/envs/tte/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hyper_param = {'lr': [1e-2, 1e-3, 1e-4], 'dropout': [0, 0.1, 0.5]}
lrs = hyper_param['lr']
dropouts = hyper_param['dropout']
saving_tags = [['lr_1e_2', 'lr_1e_3', 'lr_1e_4'], ['dp_0', 'dp_01', 'dp_05']]


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

using cuda:0 device.


In [3]:
data_transform = {
    "train": transforms.Compose([transforms.Resize((224, 224)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    "val": transforms.Compose([transforms.Resize((224, 224)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}

In [4]:
data_root ='./data'
image_path = data_root
train_dataset = datasets.ImageFolder(root=os.path.join(image_path, "train"),
                                     transform=data_transform["train"])
train_num = len(train_dataset)

# {'empty', 'occupied'}
flower_list = train_dataset.class_to_idx
cla_dict = dict((val, key) for key, val in flower_list.items())
# write dict into json file
json_str = json.dumps(cla_dict, indent=4)
with open('class_indices.json', 'w') as json_file:
    json_file.write(json_str)


In [5]:
batch_size = 32
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
print('Using {} dataloader workers every process'.format(nw))

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size, shuffle=True,
                                           num_workers=nw)

validate_dataset = datasets.ImageFolder(root=os.path.join(image_path, "val"),
                                        transform=data_transform["val"])
val_num = len(validate_dataset)
validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                              batch_size=batch_size, shuffle=False,
                                              num_workers=nw)
print("using {} images for training, {} images for validation.".format(train_num,
                                                                       val_num))


Using 8 dataloader workers every process
using 1970 images for training, 244 images for validation.


In [6]:
def train(lr, dp):
    print('Hyper Parameter: lr =', lr, ', dropout =', dp)
    net = vgg(model_name=model_name, dropout=dp, num_classes=2, init_weights=True)
    net.to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=lr)

    epochs = 30
    best_acc = 0.0
    save_path = './weights/'+model_name+'_'+saving_tags[0][i]+'_'+saving_tags[1][j]+'.pth'
#         save_path = './{}Net.pth'.format(model_name)
    train_steps = len(train_loader)
    for epoch in range(epochs):
        # train
        net.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            outputs = net(images.to(device))
            loss = loss_function(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)

        # validate
        net.eval()
        acc = 0.0  # accumulate accurate number / epoch
        with torch.no_grad():
            val_bar = tqdm(validate_loader, file=sys.stdout)
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))
                predict_y = torch.max(outputs, dim=1)[1]
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

        val_accurate = acc / val_num
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate))

        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(net.state_dict(), save_path)
#             torch.save(net.sate_dict(), save_path+str(epoch)+'.pth')
    print('Best Accuracy:', best_acc)
    print('Finished Training')

In [7]:
model_name='vgg16'
# hyper: lr; dropout
i, j = 0, 0
lr = lrs[i]
dp = dropouts[j]
train(lr, dp)

Hyper Parameter: lr = 0.01 , dropout = 0
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.65it/s]
[epoch 1] train_loss: 2051874086310.201  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.99it/s]
[epoch 2] train_loss: 318605.637  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.11it/s]
[epoch 3] train_loss: 11.146  val_accuracy: 0.816
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.81it/s]
[epoch 18] train_loss: 0.488  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.83it/s]
[epoch 19] train_loss: 0.491  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.68it/s]
[epoch 20] train_loss: 0.489  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.08it/s]
[e

In [8]:
i, j = 1, 0
lr = lrs[i]
dp = dropouts[j]
train(lr, dp)

Hyper Parameter: lr = 0.001 , dropout = 0
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.52it/s]
[epoch 1] train_loss: 5.308  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.56it/s]
[epoch 2] train_loss: 1.842  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.66it/s]
[epoch 3] train_loss: 0.491  val_accuracy: 0.816
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.63it/s]
[epoch 18] train_loss: 0.483  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.73it/s]
[epoch 19] train_loss: 0.483  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.53it/s]
[epoch 20] train_loss: 0.481  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.56it/s]
[e

In [9]:
i, j = 2, 0
lr = lrs[i]
dp = dropouts[j]
train(lr, dp)

Hyper Parameter: lr = 0.0001 , dropout = 0
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.45it/s]
[epoch 1] train_loss: 0.282  val_accuracy: 0.959
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.61it/s]
[epoch 2] train_loss: 0.135  val_accuracy: 0.984
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.70it/s]
[epoch 3] train_loss: 0.119  val_accuracy: 0.980
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.58it/s]
[epoch 18] train_loss: 0.018  val_accuracy: 0.984
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.75it/s]
[epoch 19] train_loss: 0.020  val_accuracy: 0.988
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.67it/s]
[epoch 20] train_loss: 0.026  val_accuracy: 0.992
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.52it/s]
[e

In [10]:
i, j = 0, 1
lr = lrs[i]
dp = dropouts[j]
train(lr, dp)

Hyper Parameter: lr = 0.01 , dropout = 0.1
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.54it/s]
[epoch 1] train_loss: 226724291477.581  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.66it/s]
[epoch 2] train_loss: 16.500  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.72it/s]
[epoch 3] train_loss: 2329.350  val_accuracy: 0.816
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.69it/s]
[epoch 18] train_loss: 0.485  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.66it/s]
[epoch 19] train_loss: 0.486  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.58it/s]
[epoch 20] train_loss: 0.486  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.54it/s]
[e

In [11]:
i, j = 1, 1
lr = lrs[i]
dp = dropouts[j]
train(lr, dp)

Hyper Parameter: lr = 0.001 , dropout = 0.1
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.96it/s]
[epoch 1] train_loss: 0.831  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.69it/s]
[epoch 2] train_loss: 0.483  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.68it/s]
[epoch 3] train_loss: 0.487  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.74it/s]
[epoch 18] train_loss: 0.484  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.51it/s]
[epoch 19] train_loss: 0.483  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.67it/s]
[epoch 20] train_loss: 0.484  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.74it/s]
[e

In [13]:
i, j = 2, 1
lr = lrs[i]
dp = dropouts[j]
train(lr, dp)

Hyper Parameter: lr = 0.0001 , dropout = 0.1
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.56it/s]
[epoch 1] train_loss: 0.333  val_accuracy: 0.930
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.95it/s]
[epoch 2] train_loss: 0.160  val_accuracy: 0.975
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.00it/s]
[epoch 3] train_loss: 0.119  val_accuracy: 0.967
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.52it/s]
[epoch 18] train_loss: 0.027  val_accuracy: 0.984
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.74it/s]
[epoch 19] train_loss: 0.033  val_accuracy: 0.980
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.45it/s]
[epoch 20] train_loss: 0.025  val_accuracy: 0.996
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.56it/s]
[e

In [14]:
i, j = 0, 2
lr = lrs[i]
dp = dropouts[j]
train(lr, dp)

Hyper Parameter: lr = 0.01 , dropout = 0.5
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.80it/s]
[epoch 1] train_loss: 4818895057.922  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.63it/s]
[epoch 2] train_loss: 0.710  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.69it/s]
[epoch 3] train_loss: 0.546  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.75it/s]
[epoch 18] train_loss: 0.485  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.80it/s]
[epoch 19] train_loss: 0.486  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.62it/s]
[epoch 20] train_loss: 0.485  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.55it/s]
[e

In [15]:
i, j = 1, 2
lr = lrs[i]
dp = dropouts[j]
train(lr, dp)

Hyper Parameter: lr = 0.001 , dropout = 0.5
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.56it/s]
[epoch 1] train_loss: 1.646  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.84it/s]
[epoch 2] train_loss: 0.943  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.62it/s]
[epoch 3] train_loss: 0.488  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.70it/s]
[epoch 18] train_loss: 0.484  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.98it/s]
[epoch 19] train_loss: 0.489  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.59it/s]
[epoch 20] train_loss: 0.484  val_accuracy: 0.816
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.51it/s]
[e

In [16]:
i, j = 2, 2
lr = lrs[i]
dp = dropouts[j]
train(lr, dp)

Hyper Parameter: lr = 0.0001 , dropout = 0.5
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.51it/s]
[epoch 1] train_loss: 0.291  val_accuracy: 0.975
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.58it/s]
[epoch 2] train_loss: 0.148  val_accuracy: 0.967
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.66it/s]
[epoch 3] train_loss: 0.117  val_accuracy: 0.963
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.34it/s]
[epoch 18] train_loss: 0.058  val_accuracy: 0.984
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.80it/s]
[epoch 19] train_loss: 0.058  val_accuracy: 0.988
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.56it/s]
[epoch 20] train_loss: 0.030  val_accuracy: 0.992
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  5.57it/s]
[e